<a href="https://colab.research.google.com/github/MHDBST/BERT_examples/blob/master/MyBert_document_sentiment_mask_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import pandas as pd
!pip install bert-tensorflow

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)



from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

     |████████████████████████████████| 71kB 3.2MB/s 
('TPU address is', 'grpc://10.105.211.82:8470')

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 4267554410628616746),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10981692086857205992),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 5643490144842218682),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8239004730204046674),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 7419454080405238673),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 16667992858419862505),
 _DeviceAttr

In [2]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from bert import modeling
# import optimization
# import run_classifier
from bert import run_classifier_with_tfhub
# import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_repo'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325
Receiving objects: 100% (325/325), 234.66 KiB | 3.45 MiB/s, done.
Resolving deltas: 100% (186/186), done.


W0514 05:26:56.540357 139950750201728 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
BUCKET = 'bert_example' #@param {type:"string"}
TASK = 'MASKED_DOCUMENT'
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/sentim_enti/{}/Layer_wise/last_layer/smallBERT-docLevel-seq512'.format(BUCKET,TASK)
# OUTPUT_DIR = 'gs://{}/bert-tfhub/models/nontrainable/smallBERT-docLevel-seq512'.format(BUCKET)

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

***** Model output directory: gs://bert_example/bert-tfhub/models/sentim_enti/MASKED_DOCUMENT/Layer_wise/last_layer/smallBERT-docLevel-seq512 *****


In [4]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

# BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1" ##Small Bert
# BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1" ##Big Bert
def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0514 05:27:03.886852 139950750201728 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0514 05:27:06.299200 139950750201728 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 15.0  ## Activate if ** is ACTIVATED
MAX_SEQ_LENGTH = 512
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 30
SAVE_SUMMARY_STEPS = 20
trainable=True


In [6]:
from tensorflow import keras
import os
import re


data_train = pd.read_csv('/content/train_v5.csv', encoding='latin-1')
data_dev = pd.read_csv('/content/dev_v5.csv', encoding='latin-1')
data_test = pd.read_csv('/content/random_test_v5.csv', encoding='latin-1')
data_test_fixed = pd.read_csv('/content/fixed_test_v5.csv', encoding='latin-1')




from tensorflow import keras
import os
import re

# Load all files from a directory in a DataFrame.
def load_directory_data(df,sentence_column):
  data = {}
  data["sentence"] = df[TASK] ## could be DOCUMENT, ENCLOSED_DOCUMENT, MASKED_DOCUMENT
#   data["sentence"] = df[sentence_column]
  data["sentiment"] =df["TRUE_SENTIMENT"]
#   data['document'] = df['DOCUMENT']
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(df,sentence_column='DOCUMENT',index = None):
  df_new = load_directory_data(df[:index],sentence_column)
  pos_df = df_new[df_new['sentiment'] == 'Positive']
  neg_df = df_new[df_new['sentiment'] == 'Negative']
  neu_df = df_new[df_new['sentiment'] == 'Neutral']
  pos_df["polarity"] = 1
  neg_df["polarity"] = -1
  neu_df["polarity"] = 0
  return pd.concat([pos_df, neg_df,neu_df]).sample(frac=1).reset_index(drop=True)


train = load_dataset(data_train)
test = load_dataset(data_test)
dev = load_dataset(data_dev)#,sentence_column='DOCUMENT')
test_fixed = load_dataset(data_test_fixed)
# dev_summary = load_dataset(data_summary,sentence_column='summary')



/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [0]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
label_list = [-1, 0, 1]

In [0]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

dev_InputExamples = dev.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples_fixed = test_fixed.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)



## These two lines should be activated if ** is not activated
num_train_steps = int(len(train_InputExamples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
# ITERATIONS_PER_LOOP = 1000 # I don't know what it is doing just decrease it to smaller value
ITERATIONS_PER_LOOP = int(len(train_InputExamples) / TRAIN_BATCH_SIZE) ## set as the number of iterations in each epoch 

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2,
      ),keep_checkpoint_max=15)



In [9]:
train_features = run_classifier.convert_examples_to_features(
      train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
eval_features = run_classifier.convert_examples_to_features(
      dev_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = run_classifier.convert_examples_to_features(
      test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

test_features_fixed = run_classifier.convert_examples_to_features(
      test_InputExamples_fixed, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 1501


I0514 05:27:09.794378 139950750201728 run_classifier.py:774] Writing example 0 of 1501


INFO:tensorflow:*** Example ***


I0514 05:27:09.836448 139950750201728 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0514 05:27:32.254643 139950750201728 run_classifier.py:466] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

I0514 05:27:32.259726 139950750201728 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

INFO:tensorflow:label: 1 (id = 2)


I0514 05:27:32.264044 139950750201728 run_classifier.py:468] label: 1 (id = 2)


In [0]:
# import modeling
# import run_classifier
import tokenization

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Functions and classes related to optimization (weight updates)."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import re
import tensorflow as tf


def create_optimizer(loss, init_lr, num_train_steps, num_warmup_steps, use_tpu):
  """Creates an optimizer training op."""
  global_step = tf.train.get_or_create_global_step()

  learning_rate = tf.constant(value=init_lr, shape=[], dtype=tf.float32)

  # Implements linear decay of the learning rate.
  learning_rate = tf.train.polynomial_decay(
      learning_rate,
      global_step,
      num_train_steps,
      end_learning_rate=0.0,
      power=1.0,
      cycle=False)

  # Implements linear warmup. I.e., if global_step < num_warmup_steps, the
  # learning rate will be `global_step/num_warmup_steps * init_lr`.
  if num_warmup_steps:
    global_steps_int = tf.cast(global_step, tf.int32)
    warmup_steps_int = tf.constant(num_warmup_steps, dtype=tf.int32)

    global_steps_float = tf.cast(global_steps_int, tf.float32)
    warmup_steps_float = tf.cast(warmup_steps_int, tf.float32)

    warmup_percent_done = global_steps_float / warmup_steps_float
    warmup_learning_rate = init_lr * warmup_percent_done

    is_warmup = tf.cast(global_steps_int < warmup_steps_int, tf.float32)
    learning_rate = (
        (1.0 - is_warmup) * learning_rate + is_warmup * warmup_learning_rate)

  # It is recommended that you use this optimizer for fine tuning, since this
  # is how the model was trained (note that the Adam m/v variables are NOT
  # loaded from init_checkpoint.)
  optimizer = AdamWeightDecayOptimizer(
      learning_rate=learning_rate,
      weight_decay_rate=0.01,
      beta_1=0.9,
      beta_2=0.999,
      epsilon=1e-6,
      exclude_from_weight_decay=["LayerNorm", "layer_norm", "bias"])

  if use_tpu:
    optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)

  pre_tvars = tf.trainable_variables()
#   print('all trainable variables: >>',pre_tvars)
  tvars = pre_tvars
#   tvars = [item for item in pre_tvars if not '/layer_0/'  in item.name and
#            not '/layer_1/'  in item.name and not '/layer_2/'  in item.name
#           and not '/layer_3/'  in item.name and not '/layer_4/'  in item.name and
#            not '/layer_5/'  in item.name and not '/layer_6/'  in item.name 
#           and not '/layer_7/'  in item.name and not '/layer_8/'  in item.name and 
#            not '/layer_9/'  in item.name ]
  print('excluded trainable variables: >>',tvars)
  grads = tf.gradients(loss, tvars)

  # This is how the model was pre-trained.
  (grads, _) = tf.clip_by_global_norm(grads, clip_norm=1.0)

  train_op = optimizer.apply_gradients(
      zip(grads, tvars), global_step=global_step)

  # Normally the global step update is done inside of `apply_gradients`.
  # However, `AdamWeightDecayOptimizer` doesn't do this. But if you use
  # a different optimizer, you should probably take this line out.
  new_global_step = global_step + 1
  train_op = tf.group(train_op, [global_step.assign(new_global_step)])
  return train_op


class AdamWeightDecayOptimizer(tf.train.Optimizer):
  """A basic Adam optimizer that includes "correct" L2 weight decay."""

  def __init__(self,
               learning_rate,
               weight_decay_rate=0.0,
               beta_1=0.9,
               beta_2=0.999,
               epsilon=1e-6,
               exclude_from_weight_decay=None,
               name="AdamWeightDecayOptimizer"):
    """Constructs a AdamWeightDecayOptimizer."""
    super(AdamWeightDecayOptimizer, self).__init__(False, name)

    self.learning_rate = learning_rate
    self.weight_decay_rate = weight_decay_rate
    self.beta_1 = beta_1
    self.beta_2 = beta_2
    self.epsilon = epsilon
    self.exclude_from_weight_decay = exclude_from_weight_decay

  def apply_gradients(self, grads_and_vars, global_step=None, name=None):
    """See base class."""
    assignments = []
    for (grad, param) in grads_and_vars:
      if grad is None or param is None:
        continue

      param_name = self._get_variable_name(param.name)

      m = tf.get_variable(
          name=param_name + "/adam_m",
          shape=param.shape.as_list(),
          dtype=tf.float32,
          trainable=False,
          initializer=tf.zeros_initializer())
      v = tf.get_variable(
          name=param_name + "/adam_v",
          shape=param.shape.as_list(),
          dtype=tf.float32,
          trainable=False,
          initializer=tf.zeros_initializer())

      # Standard Adam update.
      next_m = (
          tf.multiply(self.beta_1, m) + tf.multiply(1.0 - self.beta_1, grad))
      next_v = (
          tf.multiply(self.beta_2, v) + tf.multiply(1.0 - self.beta_2,
                                                    tf.square(grad)))

      update = next_m / (tf.sqrt(next_v) + self.epsilon)

      # Just adding the square of the weights to the loss function is *not*
      # the correct way of using L2 regularization/weight decay with Adam,
      # since that will interact with the m and v parameters in strange ways.
      #
      # Instead we want ot decay the weights in a manner that doesn't interact
      # with the m/v parameters. This is equivalent to adding the square
      # of the weights to the loss with plain (non-momentum) SGD.
      if self._do_use_weight_decay(param_name):
        update += self.weight_decay_rate * param

      update_with_lr = self.learning_rate * update

      next_param = param - update_with_lr

      assignments.extend(
          [param.assign(next_param),
           m.assign(next_m),
           v.assign(next_v)])
    return tf.group(*assignments, name=name)

  def _do_use_weight_decay(self, param_name):
    """Whether to use L2 weight decay for `param_name`."""
    if not self.weight_decay_rate:
      return False
    if self.exclude_from_weight_decay:
      for r in self.exclude_from_weight_decay:
        if re.search(r, param_name) is not None:
          return False
    return True

  def _get_variable_name(self, param_name):
    """Get the variable name from the tensor name."""
    m = re.match("^(.*):\\d+$", param_name)
    if m is not None:
      param_name = m.group(1)
    return param_name

In [0]:
# weight_loss = 
# neg_w = 1./len(dev_par[dev_par['label']==0])
# pos_w = 1./len(dev_par[dev_par['label']==1])
# class_weights_arr = [neg_w/(neg_w+pos_w),pos_w/(neg_w+pos_w)]
def create_model(is_training, input_ids, input_mask, segment_ids, labels,
                 num_labels, bert_hub_module_handle):
  """Creates a classification model."""
  tags = set()
  if is_training:
    tags.add("train")
  bert_module = hub.Module(bert_hub_module_handle, tags=tags, trainable=trainable)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # In the demo, we are doing a simple classification task on the entire
  # segment.
  #
  # If you want to use the token-level output, use
  # bert_outputs["sequence_output"] instead.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)# * weight_loss)

    return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps, use_tpu, bert_hub_module_handle,init_checkpoint=None):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        is_training, input_ids, input_mask, segment_ids, label_ids, num_labels,
        bert_hub_module_handle)
    
    
    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      print('initiating from checkpoint')
      cp_tvars = tvars[:]
      tvars = [var for var in cp_tvars if not 'output_weights' in var.name and   var.name!='output_bias:0']
      (assignment_map, initialized_variable_names
      ) = get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
        
        

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      train_op = create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(label_ids, predictions)
        loss = tf.metrics.mean(per_example_loss)
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
        }

      eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.PREDICT:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions={"probabilities": probabilities},scaffold_fn=scaffold_fn)
    else:
      raise ValueError(
          "Only TRAIN, EVAL and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn

In [12]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR
### Activate it if ** part is not activated 
model_fn = model_fn_builder(
    init_checkpoint= OUTPUT_DIR + '/model.ckpt-7836', #None,
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)



W0514 05:27:36.126173 139950750201728 estimator.py:1924] Estimator's model_fn (<function model_fn at 0x7f4894c43140>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.105.211.82:8470"
    }
  }
}
, '_keep_checkpoint_max': 15, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4894c06990>, '_model_dir': 'gs://bert_example/bert-tfhub/models/sentim_enti/MASKED_DOCUMENT/Layer_wise/last_layer/smallBERT-docLevel-seq512', '_protocol': None, '_save_checkpoints_steps': 30, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=46, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_cluster': <tensorflow.python.distribute.cluster

I0514 05:27:36.129734 139950750201728 estimator.py:201] Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.105.211.82:8470"
    }
  }
}
, '_keep_checkpoint_max': 15, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4894c06990>, '_model_dir': 'gs://bert_example/bert-tfhub/models/sentim_enti/MASKED_DOCUMENT/Layer_wise/last_layer/smallBERT-docLevel-seq512', '_protocol': None, '_save_checkpoints_steps': 30, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=46, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_cluster

INFO:tensorflow:_TPUContext: eval_on_tpu True


I0514 05:27:36.132102 139950750201728 tpu_context.py:202] _TPUContext: eval_on_tpu True


In [0]:
import collections
import re
import tensorflow as tf

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
  """Compute the union of the current variables and checkpoint variables."""
  assignment_map = {}
  initialized_variable_names = {}

  name_to_variable = collections.OrderedDict()
  for var in tvars:
    name = var.name
    m = re.match("^(.*):\\d+$", name)
    if m is not None:
      name = m.group(1)
    name_to_variable[name] = var

  init_vars = tf.train.list_variables(init_checkpoint)

  assignment_map = collections.OrderedDict()
  for x in init_vars:
    (name, var) = (x[0], x[1])
    if name not in name_to_variable:
      continue
    assignment_map[name] = name_to_variable[name]
    initialized_variable_names[name] = 1
    initialized_variable_names[name + ":0"] = 1

  return (assignment_map, initialized_variable_names)



In [0]:
# Train the model
# tf.logging.set_verbosity(tf.logging.INFO) #DEBUG,ERROR,FATAL,INFO,WARN
def model_train(estimator):
  # We'll set sequences to be at most 128 tokens long.
#   train_features = run_classifier.convert_examples_to_features(
#       train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_InputExamples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  md = estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))
  return md



#Evaluation and Prediction 

In [0]:
def model_eval(estimator,eval_examples=dev_InputExamples):
  # Eval the model.
#   eval_examples = dev_InputExamples#processor.get_dev_examples(TASK_DATA_DIR)
#   eval_features = run_classifier.convert_examples_to_features(
#       eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))
      
  return result

In [16]:
import numpy as np
from sklearn import metrics
labels = ["Negative","Neutral", "Positive"]
def model_predict(estimator,prediction_examples,input_features,checkpoint_path=None):
  # Make predictions on a subset of eval examples
#   prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
#   input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  if not checkpoint_path == None: 
    predictions = estimator.predict(predict_input_fn,checkpoint_path=checkpoint_path)
  else:
    predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities']) for sentence, prediction in zip(prediction_examples, predictions)]

model_train(estimator_from_tfhub)

***** Started training at 2019-05-14 05:27:36.220722 *****
  Num examples = 1501
  Batch size = 32
INFO:tensorflow:  Num steps = 690


I0514 05:27:36.221796 139950750201728 <ipython-input-14-621c767db295>:8]   Num steps = 690


INFO:tensorflow:Querying Tensorflow master (grpc://10.105.211.82:8470) for TPU system metadata.


I0514 05:27:36.387352 139950750201728 tpu_system_metadata.py:59] Querying Tensorflow master (grpc://10.105.211.82:8470) for TPU system metadata.


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:Loss for final step: 0.036507845.


I0514 05:37:07.432255 139950750201728 estimator.py:359] Loss for final step: 0.036507845.


INFO:tensorflow:training_loop marked as finished


I0514 05:37:07.435744 139950750201728 error_handling.py:93] training_loop marked as finished


***** Finished training at 2019-05-14 05:37:07.447848 *****


In [0]:
# model_train(estimator_from_tfhub)

In [0]:
# tf.logging.set_verbosity(tf.logging.INFO)
# pd = model_predict(estimator_from_tfhub,train_InputExamples,train_features)#,checkpoint_path=OUTPUT_DIR+'/model.ckpt-690')
# labels_val = []
# true_label = list(train['sentiment'])
# for item in pd:
#     labels_val.append(labels[np.argmax(item[1])])
# print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
# print(metrics.classification_report(y_pred=labels_val,y_true = true_label))



tf.logging.set_verbosity(tf.logging.FATAL)
i = 1
for ckpt in range(46,700,46):
  print('Evaluating model in batch: %d'%i)
  print('Train Results---------------------------')
  pd = model_predict(estimator_from_tfhub,train_InputExamples,train_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-%d'%ckpt)
  labels_val = []
  true_label = list(train['sentiment'])
  for item in pd:
      labels_val.append(labels[np.argmax(item[1])])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  print('Dev Results---------------------------')
  pd = model_predict(estimator_from_tfhub,dev_InputExamples,eval_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-%d'%ckpt)
  labels_val = []
  true_label = list(dev['sentiment'])
  for item in pd:
      labels_val.append(labels[np.argmax(item[1])])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  print('Fixed Test Results---------------------------')
  pd = model_predict(estimator_from_tfhub,test_InputExamples_fixed,test_features_fixed,checkpoint_path=OUTPUT_DIR+'/model.ckpt-%d'%ckpt)
  labels_val = []
  true_label = list(test_fixed['sentiment'])
  for item in pd:
      labels_val.append(labels[np.argmax(item[1])])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  print('Random Test Results---------------------------')
  pd = model_predict(estimator_from_tfhub,test_InputExamples,test_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-%d'%ckpt)
  labels_val = []
  true_label = list(test['sentiment'])
  for item in pd:
      labels_val.append(labels[np.argmax(item[1])])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  i += 1

Evaluating model in batch: 1
Train Results---------------------------
initiating from checkpoint
[[  0   0 145]
 [  3   2 438]
 [  1   1 911]]
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       145
     Neutral       0.67      0.00      0.01       443
    Positive       0.61      1.00      0.76       913

   micro avg       0.61      0.61      0.61      1501
   macro avg       0.43      0.33      0.26      1501
weighted avg       0.57      0.61      0.46      1501

Dev Results---------------------------
initiating from checkpoint
[[  0   0  23]
 [  0   0  97]
 [  0   0 164]]
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        23
     Neutral       0.00      0.00      0.00        97
    Positive       0.58      1.00      0.73       164

   micro avg       0.58      0.58      0.58       284
   macro avg       0.19      0.33      0.24       284
weighted avg       0.33      0.58     

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


initiating from checkpoint
[[  0   0  50]
 [  0   0 116]
 [  0   0 174]]
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        50
     Neutral       0.00      0.00      0.00       116
    Positive       0.51      1.00      0.68       174

   micro avg       0.51      0.51      0.51       340
   macro avg       0.17      0.33      0.23       340
weighted avg       0.26      0.51      0.35       340

Random Test Results---------------------------
initiating from checkpoint
[[  0   0  28]
 [  0   0  87]
 [  0   0 170]]
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        28
     Neutral       0.00      0.00      0.00        87
    Positive       0.60      1.00      0.75       170

   micro avg       0.60      0.60      0.60       285
   macro avg       0.20      0.33      0.25       285
weighted avg       0.36      0.60      0.45       285

Evaluating model in batch: 2
Train Results---

In [42]:
pd = model_predict(estimator_from_tfhub,dev_InputExamples,eval_features,checkpoint_path=OUTPUT_DIR+ '/model.ckpt-230')
labels_val = []
true_label = list(dev['sentiment'])
for item in pd:
    labels_val.append(labels[np.argmax(item[1])])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

INFO:tensorflow:Calling model_fn.


I0513 21:28:08.328967 140458589075328 estimator.py:1111] Calling model_fn.


INFO:tensorflow:*** Features ***


I0513 21:28:09.286730 140458589075328 <ipython-input-11-6254de2efdc9>:58] *** Features ***


INFO:tensorflow:  name = input_ids, shape = (1, 512)


...

INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


I0513 21:28:41.720849 140458589075328 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:prediction_loop marked as finished


I0513 21:28:41.915715 140458589075328 error_handling.py:93] prediction_loop marked as finished


[[  2  11  10]
 [  2  18  77]
 [  4  21 139]]
              precision    recall  f1-score   support

    Negative       0.25      0.09      0.13        23
     Neutral       0.36      0.19      0.24        97
    Positive       0.62      0.85      0.71       164

   micro avg       0.56      0.56      0.56       284
   macro avg       0.41      0.37      0.36       284
weighted avg       0.50      0.56      0.51       284



In [0]:
## Run it if you want to train for a range of epochs and see the validation error and save the prediction on than
## Part Name **
tf.logging.set_verbosity(tf.logging.FATAL)
mds = []
evs = []
pds = []
# tf.logging.set_verbosity(tf.logging.INFO)
from shutil import copyfile
max_f1 = 0
true_label = list(dev['sentiment'])
for i in range(1,15):

  NUM_TRAIN_EPOCHS = i
  
  num_train_steps = int(len(train_InputExamples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
  num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

  model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB)
  
  estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE)
  
  md = model_train(estimator_from_tfhub)
  ev = model_eval(estimator_from_tfhub)
  pd = model_predict(estimator_from_tfhub,dev_InputExamples,eval_features)
  mds.append(md)
  evs.append(ev)
  pds.append(pd)
#   print(ev)
  labels_val = []
  for item in pd:
    labels_val.append(labels[np.argmax(item[1])])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  print('----------------------------next epoch: %d----------------------------'%(i+1))




for pd,ev in zip(pds,evs):
#   print(ev)
  labels_val = []
  for item in pd:
    labels_val.append(labels[np.argmax(item[1])])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

***** Started training at 2019-05-13 20:40:26.047287 *****
  Num examples = 1501
  Batch size = 32


In [0]:

# labels_val = []
# for item in predictions:
#   labels_val.append(labels[np.argmax(item[1])])
# true_label = list(dev['sentiment'])
# print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
# print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
# predictions = model_predict(estimator_from_tfhub,dev_InputExamples)
# labels_val = []
# for item in predictions:
#   labels_val.append(labels[np.argmax(item[1])])
# true_label = list(dev['sentiment'])
# print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
# print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

In [0]:
for ckpt in range(920,921,46):
  predictions = model_predict(estimator_from_tfhub,train_InputExamples)#,checkpoint_path=OUTPUT_DIR+'/model.ckpt-%d'%ckpt)
  labels_val = []
  for item in predictions:
    labels_val.append(labels[np.argmax(item[1])])
  true_label = list(train['sentiment'])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))




[[  0  99  46]
 [  0 216 227]
 [  0 116 797]]
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       145
     Neutral       0.50      0.49      0.49       443
    Positive       0.74      0.87      0.80       913

   micro avg       0.67      0.67      0.67      1501
   macro avg       0.42      0.45      0.43      1501
weighted avg       0.60      0.67      0.63      1501



INFO:tensorflow:Writing example 0 of 284


I0501 21:37:58.855226 140091562080128 run_classifier.py:774] Writing example 0 of 284


INFO:tensorflow:*** Example ***


I0501 21:37:58.865693 140091562080128 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


I0501 21:38:28.123375 140091562080128 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:prediction_loop marked as finished


I0501 21:38:28.286058 140091562080128 error_handling.py:93] prediction_loop marked as finished


[[ 9 14  0]
 [25 33 39]
 [16 50 98]]
              precision    recall  f1-score   support

    Negative       0.18      0.39      0.25        23
     Neutral       0.34      0.34      0.34        97
    Positive       0.72      0.60      0.65       164

   micro avg       0.49      0.49      0.49       284
   macro avg       0.41      0.44      0.41       284
weighted avg       0.54      0.49      0.51       284



In [0]:
ckpt = 690
df_doc = dev.copy()
predictions = model_predict(estimator_from_tfhub,dev_InputExamples,checkpoint_path=OUTPUT_DIR+'/model.ckpt-%d'%ckpt)
labels_val = []
for item in predictions:
    labels_val.append(labels[np.argmax(item[1])])
df_doc['doc_predicted'] = labels_val
true_label = list(dev['sentiment'])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))



INFO:tensorflow:Writing example 0 of 284


I0501 21:40:33.245764 140091562080128 run_classifier.py:774] Writing example 0 of 284


INFO:tensorflow:*** Example ***


I0501 21:40:33.262032 140091562080128 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


I0501 21:41:04.523036 140091562080128 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:prediction_loop marked as finished


I0501 21:41:04.697737 140091562080128 error_handling.py:93] prediction_loop marked as finished


[[ 11   9   3]
 [  9  34  54]
 [  9  30 125]]
              precision    recall  f1-score   support

    Negative       0.38      0.48      0.42        23
     Neutral       0.47      0.35      0.40        97
    Positive       0.69      0.76      0.72       164

   micro avg       0.60      0.60      0.60       284
   macro avg       0.51      0.53      0.52       284
weighted avg       0.59      0.60      0.59       284



In [0]:
# # df_summary['doc_predicted']=list(df_doc['doc_predicted'])
# print(metrics.classification_report(y_pred=df_summary['doc_predicted'],y_true=df_summary['sentiment']))
# print(metrics.classification_report(y_pred=df_summary['summary_predicted'],y_true = df_summary['sentiment']))

df_summary.to_csv('summary_classified.csv',encoding='latin1')


In [0]:
# tf.logging.set_verbosity(tf.logging.DEBUG) #DEBUG,ERROR,FATAL,INFO,WARN
# predictions = model_predict(estimator_from_tfhub,test_InputExamples)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test['sentiment'])


In [0]:
####seq 128, Small BERT, Batchsize 32 for train, 8 for dev and test
# 1)I0423 16:57:01.507206 140076901431168 basic_session_run_hooks.py:249] loss = 0.40919852, step = 46
# 2)I0423 18:08:31.116359 140500355729280 basic_session_run_hooks.py:249] loss = 0.7756149, step = 92
# 4)Loss for final step: 0.6891643.
# 5)Loss for final step: 0.9730371
# 6)Loss for final step: 0.4775733
# 7)Loss for final step: 1.2802429.
# 8)Loss for final step: 0.509207
# 9)loss = 0.29262337, step = 414
# 10)Loss for final step: 0.47267017


# 2)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.8630275
#   global_step = 92
#   loss = 0.79767215
# 3)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.8630275
#   global_step = 138
#   loss = 0.79767215
# 4)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.86550355
#   global_step = 184
#   loss = 0.83540887
# 5)***** Eval results *****
#   eval_accuracy = 0.5964286
#   eval_loss = 0.88053304
#   global_step = 230
#   loss = 0.91362196
# 6)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.90498805
#   global_step = 276
#   loss = 1.0806552
# 7)***** Eval results *****
#   eval_accuracy = 0.56785715
#   eval_loss = 0.92742974
#   global_step = 322
#   loss = 1.0180835
# 8)***** Eval results *****
#   eval_accuracy = 0.5607143
#   eval_loss = 0.9436406
#   global_step = 368
#   loss = 0.9721719
# 9)***** Eval results *****
#   eval_accuracy = 0.5607143
#   eval_loss = 0.9714315
#   global_step = 414
#   loss = 0.5798999
# 10)***** Eval results *****
#   eval_accuracy = 0.54285717
#   eval_loss = 1.0072392
#   global_step = 460
#   loss = 1.053762
  
#   DEV Info:
# 1)[[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284
# 2)[[  0   9  14]
#  [  0  25  72]
#  [  0  21 143]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.45      0.26      0.33        97
#     Positive       0.62      0.87      0.73       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.36      0.38      0.35       284
# weighted avg       0.52      0.59      0.53       284
# 3)[[  0   9  14]
#  [  0  25  72]
#  [  0  21 143]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.45      0.26      0.33        97
#     Positive       0.62      0.87      0.73       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.36      0.38      0.35       284
# weighted avg       0.52      0.59      0.53       284
# 4)[[  0   8  42]
#  [  0  16 100]
#  [  0  17 157]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        50
#      Neutral       0.39      0.14      0.20       116
#     Positive       0.53      0.90      0.66       174

#    micro avg       0.51      0.51      0.51       340
#    macro avg       0.31      0.35      0.29       340
# weighted avg       0.40      0.51      0.41       340
# 5)[[  0  17   6]
#  [  0  40  57]
#  [  0  35 129]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.43      0.41      0.42        97
#     Positive       0.67      0.79      0.72       164

#    micro avg       0.60      0.60      0.60       284
#    macro avg       0.37      0.40      0.38       284
# weighted avg       0.54      0.60      0.56       284
# 6)[[  0  19   4]
#  [  0  44  53]
#  [  0  42 122]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.42      0.45      0.44        97
#     Positive       0.68      0.74      0.71       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.37      0.40      0.38       284
# weighted avg       0.54      0.58      0.56       284
# 7)[[  0  18   5]
#  [  0  46  51]
#  [  0  47 117]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.41      0.47      0.44        97
#     Positive       0.68      0.71      0.69       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.36      0.40      0.38       284
# weighted avg       0.53      0.57      0.55       284
# 8)[[  0  18   5]
#  [  0  48  49]
#  [  0  53 111]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.40      0.49      0.44        97
#     Positive       0.67      0.68      0.67       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.36      0.39      0.37       284
# weighted avg       0.53      0.56      0.54       284
# 9)[[  0  18   5]
#  [  0  48  49]
#  [  0  54 110]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.40      0.49      0.44        97
#     Positive       0.67      0.67      0.67       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.36      0.39      0.37       284
# weighted avg       0.52      0.56      0.54       284
# 10)[[  0  18   5]
#  [  0  51  46]
#  [  0  62 102]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.39      0.53      0.45        97
#     Positive       0.67      0.62      0.64       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.35      0.38      0.36       284
# weighted avg       0.52      0.54      0.52       284


In [0]:
###### Seq:256, small BERT, batch size 32 for train, 8 for test and dev
# 3) {'loss': 0.79650426, 'eval_accuracy': 0.58214283, 'eval_loss': 0.86125755, 'global_step': 138}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# 4) {'loss': 0.73700047, 'eval_accuracy': 0.5928571, 'eval_loss': 0.8223035, 'global_step': 184}
# [[  0  11  12]
#  [  0  20  77]
#  [  0  17 147]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.42      0.21      0.28        97
#     Positive       0.62      0.90      0.73       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.35      0.37      0.34       284
# weighted avg       0.50      0.59      0.52       284

# 5) {'loss': 0.71594626, 'eval_accuracy': 0.5928571, 'eval_loss': 0.82239425, 'global_step': 230}
# [[  0  19   4]
#  [  0  51  46]
#  [  0  48 116]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.43      0.53      0.47        97
#     Positive       0.70      0.71      0.70       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.38      0.41      0.39       284
# weighted avg       0.55      0.59      0.57       284

# 6) {'loss': 0.7227276, 'eval_accuracy': 0.5857143, 'eval_loss': 0.8479867, 'global_step': 276}
# [[  0  21   2]
#  [  2  51  44]
#  [  1  49 114]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.42      0.53      0.47        97
#     Positive       0.71      0.70      0.70       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.38      0.41      0.39       284
# weighted avg       0.56      0.58      0.57       284

# 7){'loss': 0.83304703, 'eval_accuracy': 0.5642857, 'eval_loss': 0.90589315, 'global_step': 322}
# [[  0  19   4]
#  [  4  40  53]
#  [  3  42 119]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.40      0.41      0.40        97
#     Positive       0.68      0.73      0.70       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.36      0.38      0.37       284
# weighted avg       0.53      0.56      0.54       284

# 8){'loss': 0.84049994, 'eval_accuracy': 0.575, 'eval_loss': 0.93640614, 'global_step': 368}
# [[  2  18   3]
#  [  5  42  50]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.22      0.09      0.12        23
#      Neutral       0.40      0.43      0.42        97
#     Positive       0.69      0.72      0.70       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.44      0.41      0.42       284
# weighted avg       0.55      0.57      0.56       284

# 9){'loss': 0.8601472, 'eval_accuracy': 0.5642857, 'eval_loss': 0.95250976, 'global_step': 414}
# [[  0  18   5]
#  [  4  37  56]
#  [  1  41 122]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.39      0.38      0.38        97
#     Positive       0.67      0.74      0.70       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.35      0.38      0.36       284
# weighted avg       0.52      0.56      0.54       284


# 10){'loss': 0.9091368, 'eval_accuracy': 0.5535714, 'eval_loss': 0.9715489, 'global_step': 460}
# [[  0  19   4]
#  [  4  37  56]
#  [  1  44 119]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.37      0.38      0.38        97
#     Positive       0.66      0.73      0.69       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.34      0.37      0.36       284
# weighted avg       0.51      0.55      0.53       284

# 11){'loss': 0.9756337, 'eval_accuracy': 0.5714286, 'eval_loss': 1.0129306, 'global_step': 506}
# [[  3  16   4]
#  [  5  36  56]
#  [  1  41 122]]
#               precision    recall  f1-score   support

#     Negative       0.33      0.13      0.19        23
#      Neutral       0.39      0.37      0.38        97
#     Positive       0.67      0.74      0.71       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.46      0.42      0.42       284
# weighted avg       0.55      0.57      0.55       284

# 12){'loss': 0.9551747, 'eval_accuracy': 0.56785715, 'eval_loss': 1.0222387, 'global_step': 552}
# [[  3  17   3]
#  [  5  38  54]
#  [  2  43 119]]
#               precision    recall  f1-score   support

#     Negative       0.30      0.13      0.18        23
#      Neutral       0.39      0.39      0.39        97
#     Positive       0.68      0.73      0.70       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.45      0.42      0.42       284
# weighted avg       0.55      0.56      0.55       284

# 13){'loss': 1.0276382, 'eval_accuracy': 0.5714286, 'eval_loss': 1.0547612, 'global_step': 598}
# [[  3  17   3]
#  [  5  36  56]
#  [  1  41 122]]
#               precision    recall  f1-score   support

#     Negative       0.33      0.13      0.19        23
#      Neutral       0.38      0.37      0.38        97
#     Positive       0.67      0.74      0.71       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.46      0.42      0.42       284
# weighted avg       0.55      0.57      0.55       284

# 14){'loss': 1.0036505, 'eval_accuracy': 0.55714285, 'eval_loss': 1.0697843, 'global_step': 644}
# [[  3  17   3]
#  [  7  36  54]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.25      0.13      0.17        23
#      Neutral       0.37      0.37      0.37        97
#     Positive       0.67      0.72      0.70       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.43      0.41      0.41       284
# weighted avg       0.54      0.55      0.54       284

# 15){'loss': 1.0295725, 'eval_accuracy': 0.5642857, 'eval_loss': 1.0809377, 'global_step': 690}
# [[  3  17   3]
#  [  5  39  53]
#  [  1  46 117]]
#               precision    recall  f1-score   support

#     Negative       0.33      0.13      0.19        23
#      Neutral       0.38      0.40      0.39        97
#     Positive       0.68      0.71      0.69       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.46      0.42      0.42       284
# weighted avg       0.55      0.56      0.55       284

# 16){'loss': 1.0889318, 'eval_accuracy': 0.54642856, 'eval_loss': 1.1056138, 'global_step': 736}
# [[  3  17   3]
#  [  8  33  56]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.23      0.13      0.17        23
#      Neutral       0.35      0.34      0.35        97
#     Positive       0.67      0.72      0.69       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.42      0.40      0.40       284
# weighted avg       0.52      0.54      0.53       284

# 17){'loss': 1.1312778, 'eval_accuracy': 0.54642856, 'eval_loss': 1.1440269, 'global_step': 782}
# [[  3  17   3]
#  [  9  33  55]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.21      0.13      0.16        23
#      Neutral       0.35      0.34      0.35        97
#     Positive       0.67      0.72      0.69       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.41      0.40      0.40       284
# weighted avg       0.52      0.54      0.53       284

# 18){'loss': 1.1436831, 'eval_accuracy': 0.55, 'eval_loss': 1.1613237, 'global_step': 828}
# [[  3  16   4]
#  [  8  35  54]
#  [  2  45 117]]
#               precision    recall  f1-score   support

#     Negative       0.23      0.13      0.17        23
#      Neutral       0.36      0.36      0.36        97
#     Positive       0.67      0.71      0.69       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.42      0.40      0.41       284
# weighted avg       0.53      0.55      0.54       284

# 19){'loss': 1.1817628, 'eval_accuracy': 0.55, 'eval_loss': 1.1834545, 'global_step': 874}
# [[  3  16   4]
#  [  8  34  55]
#  [  1  45 118]]
#               precision    recall  f1-score   support

#     Negative       0.25      0.13      0.17        23
#      Neutral       0.36      0.35      0.35        97
#     Positive       0.67      0.72      0.69       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.42      0.40      0.41       284
# weighted avg       0.53      0.55      0.53       284

# 20){'loss': 1.1921012, 'eval_accuracy': 0.54642856, 'eval_loss': 1.1976833, 'global_step': 920}
# [[  3  16   4]
#  [  9  35  53]
#  [  2  47 115]]
#               precision    recall  f1-score   support

#     Negative       0.21      0.13      0.16        23
#      Neutral       0.36      0.36      0.36        97
#     Positive       0.67      0.70      0.68       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.41      0.40      0.40       284
# weighted avg       0.53      0.54      0.53       284

# 21){'loss': 1.2514663, 'eval_accuracy': 0.53571427, 'eval_loss': 1.2244278, 'global_step': 966}
# [[  3  16   4]
#  [ 11  32  54]
#  [  2  47 115]]
#               precision    recall  f1-score   support

#     Negative       0.19      0.13      0.15        23
#      Neutral       0.34      0.33      0.33        97
#     Positive       0.66      0.70      0.68       164

#    micro avg       0.53      0.53      0.53       284
#    macro avg       0.40      0.39      0.39       284
# weighted avg       0.51      0.53      0.52       284

# 22){'loss': 1.2630298, 'eval_accuracy': 0.5321429, 'eval_loss': 1.2527977, 'global_step': 1012}
# [[  5  14   4]
#  [ 14  32  51]
#  [  9  43 112]]
#               precision    recall  f1-score   support

#     Negative       0.18      0.22      0.20        23
#      Neutral       0.36      0.33      0.34        97
#     Positive       0.67      0.68      0.68       164

#    micro avg       0.52      0.52      0.52       284
#    macro avg       0.40      0.41      0.41       284
# weighted avg       0.52      0.52      0.52       284

# 23){'loss': 1.3330169, 'eval_accuracy': 0.53571427, 'eval_loss': 1.2815608, 'global_step': 1058}
# [[  6  13   4]
#  [ 14  31  52]
#  [  9  42 113]]
#               precision    recall  f1-score   support

#     Negative       0.21      0.26      0.23        23
#      Neutral       0.36      0.32      0.34        97
#     Positive       0.67      0.69      0.68       164

#    micro avg       0.53      0.53      0.53       284
#    macro avg       0.41      0.42      0.42       284
# weighted avg       0.53      0.53      0.53       284

# 24){'loss': 1.3111317, 'eval_accuracy': 0.5321429, 'eval_loss': 1.2898273, 'global_step': 1104}
# [[  5  14   4]
#  [ 14  32  51]
#  [  9  43 112]]
#               precision    recall  f1-score   support

#     Negative       0.18      0.22      0.20        23
#      Neutral       0.36      0.33      0.34        97
#     Positive       0.67      0.68      0.68       164

#    micro avg       0.52      0.52      0.52       284
#    macro avg       0.40      0.41      0.41       284
# weighted avg       0.52      0.52      0.52       284

In [0]:
## Larg BERT seq:256, batchsize 8 for train, dev , test
# {'loss': 1.0679293, 'eval_accuracy': 0.5955882, 'eval_loss': 0.8965841, 'global_step': 93}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.0362501, 'eval_accuracy': 0.5955882, 'eval_loss': 0.88540924, 'global_step': 186}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.0290864, 'eval_accuracy': 0.5955882, 'eval_loss': 0.8812368, 'global_step': 279}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.0271251, 'eval_accuracy': 0.5955882, 'eval_loss': 0.88323754, 'global_step': 372}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.02654, 'eval_accuracy': 0.5955882, 'eval_loss': 0.8804489, 'global_step': 465}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# SAME FOR 10 EPOCHS!!!!!